### Down ảnh về


In [1]:
import json
import requests
import os

# Đọc tệp JSON
with open('all_labeled.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

# Tạo thư mục local_images nếu chưa có
os.makedirs('local_images', exist_ok=True)

# Khởi tạo danh sách ánh xạ
image_mapping = []

# Duyệt qua từng task trong JSON
for task in data:
    task_id = task['id']
    # Kiểm tra xem task có ảnh không
    if task['data'].get('has_image', 0) == 1:
        images = task['data'].get('images', [])
        # Duyệt qua từng URL ảnh
        for i, url in enumerate(images):
            if url and url.strip():  # Kiểm tra URL không rỗng
                try:
                    # Tải ảnh
                    response = requests.get(url, timeout=10)
                    if response.status_code == 200:
                        # Tạo tên file duy nhất
                        filename = f"local_images/{task_id}_{i}.jpg"
                        # Lưu ảnh vào local
                        with open(filename, 'wb') as f:
                            f.write(response.content)
                        # Thêm vào danh sách ánh xạ
                        image_mapping.append({
                            "task_id": task_id,
                            "original_url": url,
                            "local_path": filename
                        })
                    else:
                        print(f"Lỗi khi tải URL {url} cho task {task_id}: Status code {response.status_code}")
                except Exception as e:
                    print(f"Lỗi khi tải URL {url} cho task {task_id}: {str(e)}")
            else:
                print(f"Bỏ qua URL rỗng cho task {task_id}")

# Lưu tệp ánh xạ
with open('image_mapping.json', 'w', encoding='utf-8') as f:
    json.dump(image_mapping, f, ensure_ascii=False, indent=4)

print("Hoàn tất tải ảnh và tạo tệp ánh xạ!")

Hoàn tất tải ảnh và tạo tệp ánh xạ!


### tiền xử lý và lưu ảnh

In [2]:
import json
import os
import cv2
import numpy as np
from PIL import Image

def check_image_size(image_path):

    try:
        with Image.open(image_path) as img:
            return img.size  # (width, height)
    except Exception as e:
        raise ValueError(f"Lỗi khi kiểm tra kích thước file {image_path}: {e}")

def preprocess_image_for_resnet(image_path, target_size=(224, 224)):

    # Kiểm tra kích thước gốc
    original_size = check_image_size(image_path)
    print(f"Kích thước gốc của {image_path}: {original_size}")
    
    # Đọc ảnh bằng OpenCV
    image = cv2.imread(image_path)
    if image is None:
        raise ValueError(f"Không thể tải ảnh: {image_path}")
    
    # Chuyển từ BGR sang RGB
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    # Resize ảnh về 224x224
    image = cv2.resize(image, target_size)
    
    # Chuyển sang kiểu float32 và chuẩn hóa về [0, 1]
    image_float = image.astype(np.float32) / 255.0
    
    # Lưu bản sao để đảo ngược chuẩn hóa sau này
    original_image = image_float.copy()
    
    # Chuẩn hóa theo mean và std của ResNet
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    processed_image = (image_float - mean) / std
    
    return processed_image, original_image

def save_processed_image_as_jpeg(image, output_path):

    # Đảo ngược chuẩn hóa: Nhân với 255 để đưa về [0, 255]
    image = (image * 255.0).astype(np.uint8)
    
    # Chuyển từ RGB sang BGR để lưu bằng OpenCV
    image_bgr = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    
    # Lưu ảnh
    cv2.imwrite(output_path, image_bgr)

def process_and_save_images_to_jpeg(mapping_file_path='image_mapping.json', input_dir='local_images', output_dir='processed_images'):

    # Tạo thư mục lưu ảnh đã xử lý
    os.makedirs(output_dir, exist_ok=True)
    
    # Đọc tệp ánh xạ
    with open(mapping_file_path, 'r', encoding='utf-8') as f:
        image_mapping = json.load(f)
    
    # Khởi tạo danh sách ánh xạ mới
    new_mapping = []
    
    # Duyệt qua từng mục trong image_mapping
    for entry in image_mapping:
        task_id = entry['task_id']
        original_url = entry['original_url']
        local_path = entry['local_path']
        
        # Kiểm tra xem ảnh có tồn tại không
        if not os.path.exists(local_path):
            print(f"Ảnh không tồn tại: {local_path}")
            continue
        
        try:
            # Tiền xử lý ảnh
            processed_image, original_image = preprocess_image_for_resnet(local_path)
            
            # Tạo đường dẫn lưu ảnh đã xử lý
            processed_filename = f"{task_id}_{os.path.basename(local_path).split('.')[0]}.jpeg"
            processed_path = os.path.join(output_dir, processed_filename)
            
            # Lưu ảnh dưới dạng .jpeg
            save_processed_image_as_jpeg(original_image, processed_path)
            
            # Thêm vào ánh xạ mới
            new_mapping.append({
                'task_id': task_id,
                'original_url': original_url,
                'local_path': local_path,
                'processed_path': processed_path
            })
        except ValueError as e:
            print(f"Lỗi khi xử lý ảnh {local_path}: {e}")
    
    # Lưu ánh xạ mới vào new_image_mapping.json
    with open('new_image_mapping.json', 'w', encoding='utf-8') as f:
        json.dump(new_mapping, f, ensure_ascii=False, indent=4)
    
    return new_mapping

# Ví dụ sử dụng
if __name__ == "__main__":
    new_mapping = process_and_save_images_to_jpeg()
    print(f"Đã xử lý và lưu {len(new_mapping)} ảnh vào thư mục processed_images.")
    print("Tệp ánh xạ mới đã được lưu tại: new_image_mapping.json")

Kích thước gốc của local_images/211_0.jpg: (960, 1280)
Kích thước gốc của local_images/215_0.jpg: (1440, 1920)
Kích thước gốc của local_images/215_1.jpg: (1440, 1920)
Kích thước gốc của local_images/215_2.jpg: (1440, 1920)
Kích thước gốc của local_images/217_0.jpg: (1500, 1124)
Kích thước gốc của local_images/217_1.jpg: (1124, 1500)
Kích thước gốc của local_images/218_0.jpg: (1080, 498)
Kích thước gốc của local_images/224_0.jpg: (1124, 1500)
Kích thước gốc của local_images/228_0.jpg: (1440, 1920)
Kích thước gốc của local_images/228_1.jpg: (1440, 1920)
Kích thước gốc của local_images/228_2.jpg: (1440, 1920)
Kích thước gốc của local_images/232_0.jpg: (478, 637)
Kích thước gốc của local_images/234_0.jpg: (1080, 810)
Kích thước gốc của local_images/237_0.jpg: (478, 637)
Kích thước gốc của local_images/239_0.jpg: (1080, 1080)
Kích thước gốc của local_images/239_1.jpg: (1080, 1080)
Kích thước gốc của local_images/241_0.jpg: (478, 637)
Kích thước gốc của local_images/242_0.jpg: (1080, 1080)
K